<a href="https://colab.research.google.com/github/takatoshiinaoka/google_colaboratory/blob/main/osakana_ja_to_fakeEn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jaconv
!pip install unidic
!python -m unidic download
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=eed0e14090c50dffa5e5e250bfca92c504d23be7f19a94b7114d6b2c80b09dc5
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=ea66e8fb3b3a9de8074715374c6f824f842b163ce701dd6cf59532e5055057bd
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
Successfully built unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:08<00:00, 60.4MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /us

In [2]:
import pandas as pd # データフレーム型を使えるようにするモジュール
import requests  # APIを呼ぶモジュール
import jaconv # カタカナひらがな変換
import MeCab #分かち書き
import unidic #辞書
import json

In [3]:
# fake_englishカラム追加 & カラムの並び替え
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/osakana_nazono_katakana/same-info0717.csv")
df = pd.read_json("/content/drive/MyDrive/NewOsakana.json")
df

,pk,class,name_japanese,name_english,explanation,imageUrl
0,fishes,shark,アイザメ,Dwarf gulper shark,日本の南部からオマーンにかけての深海に生息する、全長およそ1mの海水魚。,https://zukan.com/media/leaf/original/120321.j...
1,fishes,fish,アオウミガメ,Green turtle,アオウミガメの体内の脂肪が「青い（緑色）」ことからこの名前で呼ばれている。海草・海藻食である...,https://upload.wikimedia.org/wikipedia/commons...
2,fishes,shark,アオザメ,Shortfin mako shark,英名の「mako」 とは、マオリ語で「サメ」の意味です。また、短い胸鰭をもつので“Short...,https://upload.wikimedia.org/wikipedia/commons...
3,fishes,fish,アオリイカ,Bigfin reef squid,胴は丸みを帯び、胴の縁に渡って半円形のひれを持つ。外見はコウイカに似るが、甲は薄くて透明な軟...,https://upload.wikimedia.org/wikipedia/commons...
4,fishes,fish,アカウミガメ,Loggerhead turtle,他のウミガメよりも大きな頭部で、背中の甲羅は赤褐色をしていますが老成すると赤みが失われて灰褐...,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...
204,fishes,shark,ヨロイザメ,Seal shark,肝臓には豊富な肝油が含まれ、水深の深いところに潜る際に役立っている。細長い体や、とても短く丸...,https://upload.wikimedia.org/wikipedia/commons...
205,fishes,fish,ルリスズメダイ,Blue damselfish,浅いサンゴ礁域で最も普通に見られる青色の鮮やかな魚。尾ビレの色で雌雄の判別ができる。雄の尾ビ...,https://churaumi.okinawa/userfiles/fish_images...
206,fishes,shark,レモンザメ,Sicklefin lemon shark,ふっくらとした体型をしており、ゆっくりと泳ぐサメで、名前の由来はレモンのように黄色の体色をし...,https://cdn-ak.f.st-hatena.com/images/fotolife...
207,fishes,shark,ワニグチツノザメ,viper dogfish,主に日本近海から得られており、日中は水深300mの深海に潜み、夜間には表層に浮上する。最大で...,https://upload.wikimedia.org/wikipedia/commons...


In [11]:
df['fake_english']  = ""
df =df.reindex(columns=['pk', 'class', 'name_japanese','name_english', 'fake_english', 'explanation', 'imageUrl'])
df.head(2)

,pk,class,name_japanese,name_english,fake_english,explanation,imageUrl
0,fishes,shark,アイザメ,Dwarf gulper shark,,日本の南部からオマーンにかけての深海に生息する、全長およそ1mの海水魚。,https://zukan.com/media/leaf/original/120321.j...
1,fishes,fish,アオウミガメ,Green turtle,,アオウミガメの体内の脂肪が「青い（緑色）」ことからこの名前で呼ばれている。海草・海藻食である...,https://upload.wikimedia.org/wikipedia/commons...


In [5]:
# カタカナひらがな変換

word1 = "リンゴ"
change_word1 = jaconv.kata2hira(word1)
# ⇒りんご

word2 = "ぷりん"
change_word2 = jaconv.hira2kata(word2)
# ⇒プリン

print(change_word1, change_word2)

りんご プリン


In [6]:
def wakatigaki(h):
  hiragana = h
  katakana=""
  # tagger = MeCab.Tagger()
  wakati = MeCab.Tagger('-Owakati')
  result = wakati.parse(hiragana).rstrip() #ひらがなを分かち書き
  for i in range(len(result)):
    katakana+=jaconv.hira2kata(result[i]) #一文字ずつひらがなをカタカナに変換
  return katakana

In [7]:
wakatigaki("ねこざめ")

'ネ コ ザメ'

In [8]:
text="ね こ ざめ"
data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?text={text}&source=ja&target=en')
data.json()['text']

'Cat Zame'

In [12]:
for i in range(len(df)):
  word = jaconv.kata2hira(df['name_japanese'][i]) #カタカナを平仮名に変換
  text = wakatigaki(word) # ひらがなを分かち書きしてカタカナに変換
  data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?text={text}&source=ja&target=en')
  df['fake_english'][i] = data.json()['text']

  print( i,"：" ,text ," : (",data.json()['text'],")")

0 ： アイ ザメ  : ( Eye shark )
1 ： ア オ ウミガメ  : ( Sea turtle )
2 ： アオザメ  : ( Shortfin mako shark )
3 ： ア オリ イカ  : ( Aori squid )
4 ： アカ ウミガメ  : ( Loggerhead turtle )
5 ： アカ オビ ハ ナダ イ  : ( Akaobi Hanadai )
6 ： アカ クラゲ  : ( Red jellyfish )
7 ： アカシ ュモクザメ  : ( Red shark )
8 ： アカ ヒメジ  : ( Red goatfish )
9 ： アカ マツ カ サ  : ( Red pine pine )
10 ： アカ マツ カ サ  : ( Red pine pine )
11 ： アカメ  : ( Akame )
12 ： ア カ モ ン ガラ  : ( Akamon Gala )
13 ： アケボノ チョウチョウ ウ オ  : ( Akebono Butterfly Wo )
14 ： アブラ ツノザメ  : ( Squaliform Shark )
15 ： ア ブラ ボテ  : ( Abrabote )
16 ： アミメ ハギ  : ( Reticulated python )
17 ： アヤメ エビス  : ( Ayame Ebisu )
18 ： ア ラビ アン カ ー ペッ ト シャー ク  : ( Arabian Capet Shah )
19 ： アラフ ラ オオセ  : ( Arafla wobbegong )
20 ： アリアケ ギ バチ  : ( Ariake Gibachi )
21 ： イケ カツ オ  : ( Ike cutlet )
22 ： イサキ  : ( Grunt )
23 ： イシ ガ キフ グ  : ( Ishigakifugu )
24 ： イ ズ ハナ トラ ザメ  : ( Izuhana tiger shark )
25 ： イタチ ザメ  : ( Tiger shark )
26 ： イチ ハラ ビロウド ザメ  : ( Ichihara Velvet Shark )
27 ： イ ヌ ザメ  : ( Inu shark )
28 ： ウチ ワシ ュモクザメ  : (

In [13]:
df

,pk,class,name_japanese,name_english,fake_english,explanation,imageUrl
0,fishes,shark,アイザメ,Dwarf gulper shark,Eye shark,日本の南部からオマーンにかけての深海に生息する、全長およそ1mの海水魚。,https://zukan.com/media/leaf/original/120321.j...
1,fishes,fish,アオウミガメ,Green turtle,Sea turtle,アオウミガメの体内の脂肪が「青い（緑色）」ことからこの名前で呼ばれている。海草・海藻食である...,https://upload.wikimedia.org/wikipedia/commons...
2,fishes,shark,アオザメ,Shortfin mako shark,Shortfin mako shark,英名の「mako」 とは、マオリ語で「サメ」の意味です。また、短い胸鰭をもつので“Short...,https://upload.wikimedia.org/wikipedia/commons...
3,fishes,fish,アオリイカ,Bigfin reef squid,Aori squid,胴は丸みを帯び、胴の縁に渡って半円形のひれを持つ。外見はコウイカに似るが、甲は薄くて透明な軟...,https://upload.wikimedia.org/wikipedia/commons...
4,fishes,fish,アカウミガメ,Loggerhead turtle,Loggerhead turtle,他のウミガメよりも大きな頭部で、背中の甲羅は赤褐色をしていますが老成すると赤みが失われて灰褐...,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...
204,fishes,shark,ヨロイザメ,Seal shark,Yoroi shark,肝臓には豊富な肝油が含まれ、水深の深いところに潜る際に役立っている。細長い体や、とても短く丸...,https://upload.wikimedia.org/wikipedia/commons...
205,fishes,fish,ルリスズメダイ,Blue damselfish,Blue devil,浅いサンゴ礁域で最も普通に見られる青色の鮮やかな魚。尾ビレの色で雌雄の判別ができる。雄の尾ビ...,https://churaumi.okinawa/userfiles/fish_images...
206,fishes,shark,レモンザメ,Sicklefin lemon shark,Lemon shark,ふっくらとした体型をしており、ゆっくりと泳ぐサメで、名前の由来はレモンのように黄色の体色をし...,https://cdn-ak.f.st-hatena.com/images/fotolife...
207,fishes,shark,ワニグチツノザメ,viper dogfish,Viper dogfish Squaliform shark,主に日本近海から得られており、日中は水深300mの深海に潜み、夜間には表層に浮上する。最大で...,https://upload.wikimedia.org/wikipedia/commons...


In [14]:
path = '/content/drive/MyDrive/add_fake_osakana.json'
df.to_json(path,orient='records',force_ascii=False)

# Cat Shark 頑張った

In [11]:
text="ネコザメ"# ネコザメ を翻訳

data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?\
text={text}&source=ja&target=en')

data.json()['text'] #レスポンスを表示

'Bullhead shark'

In [12]:
text = jaconv.kata2hira(text) #平仮名に変換

print("text:",text) # text を翻訳
data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?\
text={text}&source=ja&target=en')

data.json()['text'] #レスポンスを表示

text: ねこざめ


'Bullhead shark'

In [13]:
wakati = MeCab.Tagger('-Owakati') #分かち書き
result = wakati.parse(text).rstrip()


print("result:",result) # result を翻訳
data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?\
text={result}&source=ja&target=en')

data.json()['text'] #レスポンスを表示

result: ね こ ざめ


'Cat Zame'

In [14]:
katakana=""

for i in range(len(result)):
  katakana+=jaconv.hira2kata(result[i]) #一文字ずつひらがなをカタカナに変換

print("katakana:",katakana) # katakana を翻訳
data = requests.get(f'https://script.google.com/macros/s/AKfycbywady3Xqrw9esqFl85YtROC5UmfaCyf7p2H8QdxmhLoXsQVXBft_PmSbEEcGZKbYdW/exec?\
text={katakana}&source=ja&target=en')

data.json()['text'] #レスポンスを表示

katakana: ネ コ ザメ


'Cat shark'